In [1]:
# TJ needed imports
import sys
import os
#TJ change directory to include the entire ASTRO5160 directory
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) 
from Week8.Class16_notebook import *
from astropy.coordinates import SkyCoord
from scipy.spatial import cKDTree
import numpy as np
from astropy import units as u
import glob

In [6]:
#python task #1 Find object closest to (188.53667, 21.04572)
sweep_directory = '/d/scratch/ASTR5160/data/legacysurvey/dr9/south/sweep/9.0' #TJ assign necessary sweep file directory
loc = SkyCoord(188.53667, 21.04572, unit = u.deg)
sweep_file = search_directory_for_location(sweep_directory, loc)

loc_xyz = loc.cartesian.xyz.T
tree = cKDTree() #TJ build ckd tree for efficient searching

#TJ query the tree for the nearest neighbor of each object in match_all_these table
_, matched_indices = tree.query(obs_xyz) #TJ first output is the matched entry's euclidean distance from source, not easy to convert to deg


TypeError: __init__() takes at least 1 positional argument (0 given)

In [7]:
sweep_file

[]